In [1]:
import pandas as pd
import numpy as np
from causalimpact import CausalImpact
import csv
import sys
from datetime import datetime
import dtw
from sklearn import preprocessing

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



principal variables:
intervention date in datetime
date start in datetime
date end in datetime
metric in str
test_and_control set in csv or xslx, 
dataset in csv or xslx,
  optional:
  alpha
  events per season
  number of seasons
  backtest periods
  standardize

actions on fit():
check column names
check all dates are type datetime
sort by date
rename BestControl if found
check for issues with page names
append 'test' to page column
merge
pivot
find partition 1 and 2
drop date
check column order, find and place 'test' column
run causal impact with data imputed above
    returns:
    CausalImpact object

In [16]:
pdp = pd.read_excel('C:/Users/akiro/Documents/Adidas/USA/us_test_06_pdp_titles_categories/data/inputs/pdps.xlsx')

In [17]:
data1 = pd.read_csv('C:/Users/akiro/Documents/Adidas/USA/us_test_06_pdp_titles_categories/data/inputs/us_2021_11.csv')
data2 = pd.read_csv('C:/Users/akiro/Documents/Adidas/USA/us_test_06_pdp_titles_categories/data/inputs/bq-all.csv')
data1 = data1.groupby(['date','page']).sum().reset_index()
data1 = data1[['date', 'page', 'impressions']]
data2 = data2[['date', 'page', 'impressions']]

In [18]:
data = pd.concat([data1,data2], ignore_index=True).sort_values('date')

In [54]:
def fit(int_time, start, end, distances, 
        causal_input, alpha = 0.95, events_per_season = 1, 
        seasons = 1, btest = 0, standardize = False):
    
    distances = distances.append({'index':'TEST'}, ignore_index=True)
    
    pvt_longer = causal_input.merge(distances['index'], how='inner', on=['page']).drop_duplicates().sort_values('date').reset_index(drop=True)
    
    raw_ts = pvt_longer.pivot_table(index='date', columns='page', values='value', aggfunc='sum').reset_index()
    
    partition1 = raw_ts[raw_ts.date == '2021-11-12'].index.values.astype(int)[0]
    
    partition2 = raw_ts[raw_ts.date == '2021-11-24'].index.values.astype(int)[0]
    
    pre_period = [0,  partition1 - 1]
    
    post_period = [partition1, 324]
    
    final = raw_ts.[['value','page']]
    
    ci = CausalImpact(final, pre_period, post_period, alpha=0.05, model_args = {'nseasons': 14,'season_duration': 2})

    print(ci.summary())

    ci.plot()
    
    return  self